<a href="https://colab.research.google.com/github/KaifAhmad1/code-test/blob/main/Face_Similarity_Matching.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Reverse Image Search System for Defensive Forensics**

In [1]:
!pip install -q torch transformers langchain langgraph numpy pillow requests vllm langchain_community groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 113.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 64.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 90.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.4/151.4 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import asyncio
import os
import torch
import numpy as np
import requests
from io import BytesIO
from PIL import Image, ImageEnhance, ImageFilter
import cv2
import matplotlib.pyplot as plt
import networkx as nx
import aiohttp
import time
from typing import Dict, List, Any, Tuple, Optional, Union
import json

# For embeddings and multimodal processing
from transformers import CLIPProcessor, CLIPModel, AutoProcessor, AutoModel
import torch.nn.functional as F

# For LLM orchestration
from groq import Groq
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_core.messages import SystemMessage, HumanMessage

# For graph-based agent orchestration
from langgraph.graph import StateGraph, END

In [3]:
import getpass

# Securely get API keys from user input
GROQ_API_KEY = getpass.getpass("Enter your GROQ API Key: ")
GOOGLE_CSE_ID = getpass.getpass("Enter your Google CSE ID: ")
GOOGLE_API_KEY = getpass.getpass("Enter your Google API Key: ")

Enter your GROQ API Key: ··········
Enter your Google CSE ID: ··········
Enter your Google API Key: ··········


In [4]:
# Initialize global models
print("Loading models...")
clip_model = CLIPModel.from_pretrained("openai/clip-vit-large-patch14")
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-large-patch14")

# Load BLIP-2 for better image understanding
blip_processor = AutoProcessor.from_pretrained("Salesforce/blip2-opt-2.7b")
blip_model = AutoModel.from_pretrained("Salesforce/blip2-opt-2.7b", torch_dtype=torch.float16)

# Load DINOv2 for more robust feature extraction
dinov2_model = AutoModel.from_pretrained("facebook/dinov2-base")
dinov2_processor = AutoProcessor.from_pretrained("facebook/dinov2-base")

# Set device (use GPU if available)
device = "cuda" if torch.cuda.is_available() else "cpu"
clip_model = clip_model.to(device)
blip_model = blip_model.to(device)
dinov2_model = dinov2_model.to(device)

print(f"Models loaded on {device}")

# Initialize Groq client
client = Groq(
    api_key=GROQ_API_KEY
)

# Initialize llama_llm using chat completion setup
llama_llm = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Initialize the model.",
        }
    ],
    model="llama-3.3-70b-versatile",  # Specify the model here
    temperature=0.1
)

Loading models...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/4.52k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.71G [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/548 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/436 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.48, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


Models loaded on cuda


In [5]:
######################################
# 1. IMAGE PROCESSING FUNCTIONS
######################################

async def fetch_image_async(image_url: str) -> bytes:
    """Asynchronously fetch an image from a URL"""
    async with aiohttp.ClientSession() as session:
        async with session.get(image_url) as response:
            if response.status != 200:
                raise ValueError(f"Failed to fetch image: {response.status}")
            return await response.read()

async def preprocess_image_async(image_url_or_bytes: Union[str, bytes]) -> Image.Image:
    """
    Download and preprocess the image asynchronously.
    Enhances contrast, sharpens, and returns a clean RGB image.
    """
    if isinstance(image_url_or_bytes, str):
        image_bytes = await fetch_image_async(image_url_or_bytes)
    else:
        image_bytes = image_url_or_bytes

    image = Image.open(BytesIO(image_bytes)).convert("RGB")

    # Apply image enhancements
    enhancer = ImageEnhance.Contrast(image)
    image = enhancer.enhance(1.5)
    image = image.filter(ImageFilter.SHARPEN)

    # Resize for consistent processing
    if max(image.size) > 1024:
        image.thumbnail((1024, 1024), Image.LANCZOS)

    return image

def detect_faces(image: Image.Image) -> List[np.ndarray]:
    """
    Detect faces in an image and return face embeddings.
    """
    # Convert PIL image to OpenCV format
    img_cv = np.array(image)
    img_cv = img_cv[:, :, ::-1].copy()  # RGB to BGR

    # Load face detection model (Haar Cascade for simplicity)
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

    # Detect faces
    gray = cv2.cvtColor(img_cv, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.1, 4)

    face_images = []
    for (x, y, w, h) in faces:
        # Extract and preprocess each face
        face_img = image.crop((x, y, x+w, y+h))
        face_img = face_img.resize((224, 224), Image.LANCZOS)
        face_images.append(face_img)

    return face_images

In [6]:
######################################
# 2. EMBEDDING GENERATION FUNCTIONS
######################################

async def generate_clip_embedding(image: Image.Image) -> np.ndarray:
    """
    Generate a robust image embedding using the CLIP model.
    """
    inputs = clip_processor(images=image, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = clip_model.get_image_features(**inputs)

    # Normalize embedding
    embedding = outputs.cpu().numpy()
    norm = np.linalg.norm(embedding)
    return embedding / norm if norm > 0 else embedding

async def generate_blip_embedding(image: Image.Image) -> np.ndarray:
    """
    Generate an embedding using BLIP-2 for better semantic understanding.
    """
    inputs = blip_processor(images=image, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = blip_model.get_image_features(**inputs)

    embedding = outputs.cpu().numpy()
    norm = np.linalg.norm(embedding)
    return embedding / norm if norm > 0 else embedding

async def generate_dinov2_embedding(image: Image.Image) -> np.ndarray:
    """
    Generate an embedding using DINOv2 for better feature representation.
    """
    inputs = dinov2_processor(images=image, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = dinov2_model(**inputs).last_hidden_state[:, 0]  # Use CLS token

    embedding = outputs.cpu().numpy()
    norm = np.linalg.norm(embedding)
    return embedding / norm if norm > 0 else embedding

async def generate_image_description(image: Image.Image) -> str:
    """
    Generate a detailed description of the image using BLIP-2.
    """
    prompt = "Describe this image in detail, focusing on any identifiable people, locations, or objects:"
    inputs = blip_processor(images=image, text=prompt, return_tensors="pt").to(device)

    with torch.no_grad():
        outputs = blip_model.generate(**inputs, max_new_tokens=100)

    return blip_processor.batch_decode(outputs, skip_special_tokens=True)[0]

async def extract_classical_features(image: Image.Image) -> np.ndarray:
    """
    Extract classical computer vision features using ORB and SIFT.
    """
    # Convert to OpenCV format
    image_np = np.array(image)
    gray = cv2.cvtColor(image_np, cv2.COLOR_RGB2GRAY)

    # ORB features
    orb = cv2.ORB_create(nfeatures=1000)
    keypoints_orb, descriptors_orb = orb.detectAndCompute(gray, None)

    # If SIFT is available (OpenCV contrib)
    try:
        sift = cv2.SIFT_create()
        keypoints_sift, descriptors_sift = sift.detectAndCompute(gray, None)
    except:
        descriptors_sift = np.array([])

    # Combine or choose best descriptor
    if descriptors_orb is None and descriptors_sift is None:
        return np.array([])
    elif descriptors_orb is None:
        return descriptors_sift
    elif descriptors_sift is None:
        return descriptors_orb
    else:
        # Take the one with more keypoints
        return descriptors_sift if len(keypoints_sift) > len(keypoints_orb) else descriptors_orb

In [7]:
######################################
# 3. SEARCH ENGINE FUNCTIONS
######################################

async def search_private_db(embedding: np.ndarray, description: str) -> List[Dict]:
    """
    Simulated private database search using embeddings.
    In a real implementation, this would query a vector database.
    """
    # Simulate database search delay
    await asyncio.sleep(0.5)

    # Mock results - in a real system this would search a vector DB
    return [
        {"source": "Private DB", "match": "Person_123", "score": 0.91, "metadata": {"date": "2023-10-15"}},
        {"source": "Private DB", "match": "Person_456", "score": 0.85, "metadata": {"date": "2023-09-22"}}
    ]

async def search_twitter(embedding: np.ndarray, description: str) -> List[Dict]:
    """
    Simulated Twitter/X search (would use Twitter API in production).
    """
    await asyncio.sleep(0.7)

    # Keywords from description
    keywords = description.split()[:5]  # Simple keyword extraction

    return [
        {"source": "Twitter", "match": "Tweet_Image_456", "score": 0.87, "metadata": {
            "username": "@user123",
            "posted": "2023-11-01",
            "keywords": keywords
        }}
    ]

async def search_reddit(embedding: np.ndarray, description: str) -> List[Dict]:
    """
    Simulated Reddit search (would use Reddit API in production).
    """
    await asyncio.sleep(0.6)

    return [
        {"source": "Reddit", "match": "Reddit_Post_321", "score": 0.89, "metadata": {
            "subreddit": "r/pics",
            "posted": "2023-10-25"
        }}
    ]

async def search_instagram(embedding: np.ndarray, description: str) -> List[Dict]:
    """
    Simulated Instagram search (would use Instagram API in production).
    """
    await asyncio.sleep(0.8)

    return [
        {"source": "Instagram", "match": "Insta_Post_654", "score": 0.88, "metadata": {
            "username": "user456",
            "posted": "2023-11-12",
            "location": "New York"
        }}
    ]

async def search_osint_sources(embedding: np.ndarray, description: str) -> List[Dict]:
    """
    Simulated OSINT search across multiple sources.
    """
    await asyncio.sleep(1.0)

    return [
        {"source": "OSINT", "match": "DarkWeb_Post_999", "score": 0.83, "metadata": {
            "forum": "anonymous_forum",
            "date": "2023-09-10"
        }},
        {"source": "OSINT", "match": "Telegram_Group_123", "score": 0.79, "metadata": {
            "group": "public_channel_xyz",
            "date": "2023-10-30"
        }}
    ]

async def search_tineye(image: Image.Image) -> List[Dict]:
    """
    Search TinEye reverse image search API (would need API key).
    """
    # Save image to bytes for upload
    img_byte_arr = BytesIO()
    image.save(img_byte_arr, format='JPEG')
    img_byte_arr.seek(0)

    # In a real implementation, use TinEye API
    await asyncio.sleep(1.2)

    # Simulate TinEye results
    return [
        {"source": "TinEye", "match": "Website_ABC", "score": 0.92, "metadata": {
            "domain": "example.com",
            "first_crawled": "2023-08-15"
        }}
    ]

async def search_google_images(image: Image.Image, description: str) -> List[Dict]:
    """
    Search Google Images (would need API key and custom search engine ID).
    """
    # Convert description to search terms
    search_terms = " ".join(description.split()[:7])

    # In a real implementation, use Google Custom Search API
    await asyncio.sleep(1.0)

    # Simulate Google results
    return [
        {"source": "Google Images", "match": "News_Site_XYZ", "score": 0.86, "metadata": {
            "url": "https://example-news.com/article123",
            "title": "Example article related to the image"
        }}
    ]

async def merge_search_results(*results: List[Dict]) -> List[Dict]:
    """
    Merge search results from multiple sources and sort by score.
    """
    merged = []
    for result_list in results:
        merged.extend(result_list)

    # Sort by score in descending order
    return sorted(merged, key=lambda x: x.get('score', 0), reverse=True)

In [8]:
######################################
# 4. ANALYSIS FUNCTIONS
######################################

async def analyze_results_with_llm(results: List[Dict], image_description: str) -> str:
    """
    Analyze search results using an LLM to provide insights.
    """
    prompt_template = """
    You are a forensic analyst analyzing reverse image search results.

    IMAGE DESCRIPTION:
    {image_description}

    SEARCH RESULTS:
    {search_results}

    Provide a detailed forensic assessment considering:
    1. Cross-referencing of entities across sources.
    2. Temporal and geographic correlations if available.
    3. Reliability of each source and its score.
    4. Potential identity matches and their confidence.
    5. Key insights about the origin and distribution of this image.

    Format your analysis as a structured report with clear sections.
    """

    prompt = PromptTemplate(
        input_variables=["image_description", "search_results"],
        template=prompt_template
    )

    chain = LLMChain(llm=forensis_llm, prompt=prompt)
    response = await chain.arun(
        image_description=image_description,
        search_results=json.dumps(results, indent=2)
    )

    return response.strip()

async def threat_assessment(analysis: str) -> Dict[str, Any]:
    """
    Assess potential threats or concerns based on the analysis.
    """
    prompt_template = """
    Based on the following forensic image analysis, assess the potential threats or concerns:

    {analysis}

    Provide a JSON object with the following fields:
    1. "threat_level": A value from 0-10 where 0 is no concern and 10 is severe concern
    2. "categories": Array of concern categories (e.g. ["identity_theft", "misinformation", "privacy_breach"])
    3. "reasoning": Brief explanation of your assessment
    4. "recommended_actions": Array of recommended actions to take

    Format as valid JSON only, with no additional text.
    """

    prompt = PromptTemplate(
        input_variables=["analysis"],
        template=prompt_template
    )

    chain = LLMChain(llm=forensis_llm, prompt=prompt)
    response = await chain.arun(analysis=analysis)

    try:
        # Try to parse the response as JSON
        return json.loads(response.strip())
    except json.JSONDecodeError:
        # Fallback if LLM doesn't produce valid JSON
        return {
            "threat_level": 5,
            "categories": ["unknown"],
            "reasoning": "Failed to parse LLM response as JSON. Original analysis available in report.",
            "recommended_actions": ["Review the full analysis manually"]
        }

async def graph_link_analysis(results: List[Dict]) -> plt.Figure:
    """
    Create a network graph visualization of the search results.
    """
    G = nx.Graph()

    # Add nodes and edges
    for result in results:
        source = result['source']
        match = result['match']
        score = result['score']

        # Add source and match nodes
        G.add_node(source, type='source')
        G.add_node(match, type='match')

        # Add edge between source and match
        G.add_edge(source, match, weight=score)

        # Add metadata connections if available
        if 'metadata' in result:
            for key, value in result['metadata'].items():
                if isinstance(value, str):
                    meta_node = f"{key}:{value}"
                    G.add_node(meta_node, type='metadata')
                    G.add_edge(match, meta_node, weight=1.0)

    # Create layout and colors
    pos = nx.spring_layout(G, seed=42)
    node_colors = []
    for node in G.nodes():
        node_type = G.nodes[node].get('type', 'unknown')
        if node_type == 'source':
            node_colors.append('lightblue')
        elif node_type == 'match':
            node_colors.append('lightgreen')
        else:
            node_colors.append('lightgray')

    # Draw graph
    plt.figure(figsize=(12, 8))
    nx.draw(G, pos, with_labels=True, node_color=node_colors,
            edge_color='gray', font_size=8, node_size=800, alpha=0.8)

    # Add weights as edge labels
    edge_labels = {(u, v): f"{d['weight']:.2f}" for u, v, d in G.edges(data=True) if 'weight' in d}
    nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_size=7)

    plt.title("Graph & Link Analysis of Search Results")

    # Return the figure object
    return plt.gcf()

In [9]:
######################################
# 5. MULTI-AGENT SYSTEM WITH LANGGRAPH
######################################

async def build_search_system(image_url: str) -> Dict[str, Any]:
    """
    Build and execute a multi-agent system for reverse image search using LangGraph.
    """
    # Define the state schema
    state_schema = {
        "image": None,
        "image_description": "",
        "faces": [],
        "clip_embedding": None,
        "blip_embedding": None,
        "dinov2_embedding": None,
        "classical_features": None,
        "search_results": {
            "private_db": [],
            "twitter": [],
            "reddit": [],
            "instagram": [],
            "osint": [],
            "tineye": [],
            "google": []
        },
        "merged_results": [],
        "analysis": "",
        "threat_assessment": {},
        "graph": None,
        "final_report": ""
    }

    # Initialize the state graph
    workflow = StateGraph(state_schema)

    # Define the nodes

    # 1. Preprocessing Node
    async def preprocess_node(state):
        print("Preprocessing image...")
        image = await preprocess_image_async(image_url)
        faces = detect_faces(image)
        image_description = await generate_image_description(image)

        print(f"Image description: {image_description[:100]}...")
        print(f"Found {len(faces)} faces")

        return {
            **state,
            "image": image,
            "faces": faces,
            "image_description": image_description
        }

    # 2. Embedding Generation Node
    async def embedding_node(state):
        print("Generating embeddings...")
        image = state["image"]

        # Generate all embeddings in parallel
        clip_embedding, blip_embedding, dinov2_embedding, classical_features = await asyncio.gather(
            generate_clip_embedding(image),
            generate_blip_embedding(image),
            generate_dinov2_embedding(image),
            extract_classical_features(image)
        )

        return {
            **state,
            "clip_embedding": clip_embedding,
            "blip_embedding": blip_embedding,
            "dinov2_embedding": dinov2_embedding,
            "classical_features": classical_features
        }

    # 3. Search Node
    async def search_node(state):
        print("Searching across sources...")
        image = state["image"]
        clip_embedding = state["clip_embedding"]
        description = state["image_description"]

        # Search all sources in parallel
        private_results, twitter_results, reddit_results, instagram_results, osint_results, tineye_results, google_results = await asyncio.gather(
            search_private_db(clip_embedding, description),
            search_twitter(clip_embedding, description),
            search_reddit(clip_embedding, description),
            search_instagram(clip_embedding, description),
            search_osint_sources(clip_embedding, description),
            search_tineye(image),
            search_google_images(image, description)
        )

        search_results = {
            "private_db": private_results,
            "twitter": twitter_results,
            "reddit": reddit_results,
            "instagram": instagram_results,
            "osint": osint_results,
            "tineye": tineye_results,
            "google": google_results
        }

        # Merge all results
        all_results = []
        for source_results in search_results.values():
            all_results.extend(source_results)

        merged_results = sorted(all_results, key=lambda x: x.get('score', 0), reverse=True)

        print(f"Found {len(merged_results)} results across all sources")

        return {
            **state,
            "search_results": search_results,
            "merged_results": merged_results
        }

    # 4. Analysis Node
    async def analysis_node(state):
        print("Analyzing results...")
        results = state["merged_results"]
        description = state["image_description"]

        analysis = await analyze_results_with_llm(results, description)
        threat_info = await threat_assessment(analysis)
        graph = await graph_link_analysis(results)

        return {
            **state,
            "analysis": analysis,
            "threat_assessment": threat_info,
            "graph": graph
        }

    # 5. Report Generation Node
    async def report_node(state):
        print("Generating final report...")

        # Format the final report with all information
        report = f"""
        # Reverse Image Search Forensic Report

        ## Image Description
        {state['image_description']}

        ## Key Findings
        - Found {len(state['merged_results'])} matches across {len(state['search_results'])} sources
        - Detected {len(state['faces'])} faces in the image
        - Threat level: {state['threat_assessment'].get('threat_level', 'Unknown')}/10

        ## Detailed Analysis
        {state['analysis']}

        ## Threat Assessment
        - Level: {state['threat_assessment'].get('threat_level', 'Unknown')}/10
        - Categories: {', '.join(state['threat_assessment'].get('categories', ['Unknown']))}
        - Reasoning: {state['threat_assessment'].get('reasoning', 'Not available')}

        ## Recommended Actions
        {', '.join(state['threat_assessment'].get('recommended_actions', ['None specified']))}

        ## Raw Search Results
        {json.dumps(state['merged_results'][:5], indent=2)}  # Show top 5 results
        """

        return {
            **state,
            "final_report": report
        }

    # Add nodes to the graph
    workflow.add_node("preprocess", preprocess_node)
    workflow.add_node("embed", embedding_node)
    workflow.add_node("search", search_node)
    workflow.add_node("analyze", analysis_node)
    workflow.add_node("report", report_node)

    # Add edges to define the workflow
    workflow.add_edge("preprocess", "embed")
    workflow.add_edge("embed", "search")
    workflow.add_edge("search", "analyze")
    workflow.add_edge("analyze", "report")
    workflow.add_edge("report", END)

    # Set the entry point
    workflow.set_entry_point("preprocess")

    # Execute the workflow
    print("Starting reverse image search workflow...")
    result = await workflow.ainvoke({})
    print("Workflow completed!")

    return result

In [10]:
######################################
# 6. MAIN FUNCTION
######################################

async def main(image_url: str) -> Dict[str, Any]:
    """
    Main function to execute the reverse image search system.
    """
    try:
        start_time = time.time()
        print(f"Processing image: {image_url}")

        result = await build_search_system(image_url)

        end_time = time.time()
        print(f"Total processing time: {end_time - start_time:.2f} seconds")

        # Print the final report
        print("\n" + "="*50)
        print("FINAL REPORT:")
        print("="*50)
        print(result["final_report"])

        return result

    except Exception as e:
        print(f"Error: {str(e)}")
        import traceback
        traceback.print_exc()
        return {"error": str(e)}

# Example usage
if __name__ == "__main__":
    # Replace with a valid image URL for testing
    test_image_url = "https://example.com/test_image.jpg"

    # Run the async main function
    import asyncio
    result = asyncio.run(main(test_image_url))

RuntimeError: asyncio.run() cannot be called from a running event loop